## Dealing with heterozygosity in the genome assembly

Here I want, ultimately, to be able to divid the scaffolds and contigs in the genome assembly into either haploid sections of the genome, or diploid, where I know the primary and alternative contig pairs at the heterozygous positions.

In practice I am never going to get 100% of these, or get 100% of them right. But I will try my best. 

Remember we are aiming for a genome size somewhere around 4.5Gb, so anything around that I think would be a success. 